In [2]:
import pandas as pd
import numpy as np

# Function to calculate Exponential Moving Average (EMA)
def calculate_ema(data, window):
    return data['close'].ewm(span=window, adjust=False).mean()

# Function to calculate Relative Strength Index (RSI)
def calculate_rsi(data, window):
    diff = data['close'].diff()
    gain = (diff.where(diff > 0, 0)).rolling(window=window).mean()
    loss = (-diff.where(diff < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to generate trading signals
def generate_signals(data):
    ema_50 = calculate_ema(data, 50)
    ema_200 = calculate_ema(data, 200)
    data['RSI'] = calculate_rsi(data, 14)
    
    data['Signal'] = np.where((data['close'] > ema_50) & (ema_50 > ema_200) & (data['RSI'] < 30), 'Buy', '')
    data['Signal'] = np.where((data['close'] < ema_50) & (ema_50 < ema_200) & (data['RSI'] > 70), 'Sell', data['Signal'])

    return data

# Function to calculate profits
def calculate_profits(data):
    capital = 10000  # Initial capital
    position = 0     # 0: No position, 1: Long (Buy), -1: Short (Sell)
    buy_price = 0    # Price at which we bought
    total_trades = 0 # Total number of trades executed
    profit = 0       # Total profit
    
    for index, row in data.iterrows():
        if row['Signal'] == 'Buy' and position != 1:  # Buy signal and not already in long position
            if position == -1:  # If in short position, close it
                profit += sell_price - row['close']
                position = 0
                total_trades += 1
            buy_price = row['close']
            position = 1
        elif row['Signal'] == 'Sell' and position != -1:  # Sell signal and not already in short position
            if position == 1:  # If in long position, close it
                profit += row['close'] - buy_price
                position = 0
                total_trades += 1
            sell_price = row['close']
            position = -1
    
    # If still in position at the end, close it based on the last price
    if position == 1:
        profit += data.iloc[-1]['close'] - buy_price
        total_trades += 1
    elif position == -1:
        profit += sell_price - data.iloc[-1]['close']
        total_trades += 1
    
    return profit, total_trades

# Load OHLCV data from Excel file
excel_file_path = 'TSLA_1min_market_price_data.csv'
data = pd.read_csv(excel_file_path)
data['time'] = pd.to_datetime(data['time'])
data.set_index('time', inplace=True)

# Generate trading signals
data_with_signals = generate_signals(data)

# Calculate profits
profit, total_trades = calculate_profits(data_with_signals)

# Print the data with signals
print("Total Profit:", profit)
print("Total Trades Executed:", total_trades)


/var/folders/sb/zlj0j6815770nsgqbs9m4bg40000gn/T/ipykernel_5682/2007114171.py:65: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['time'] = pd.to_datetime(data['time'])


Total Profit: 143.1894
Total Trades Executed: 361


In [ ]:
output_file_path = 'C://Users//lenovo//Downloads//TSLA_signals.xlsx'
data_with_signals.to_excel(output_file_path)